<a href="https://colab.research.google.com/github/sandipanbasu/aiml-capstone/blob/master/Create_Train_Test_Val.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Create Train Test and Validation data

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
import warnings
import pickle
import numpy as np
import pandas as pd
import json
from sklearn.model_selection import train_test_split
import pprint


In [9]:
# we will store the params as we go along in this object
model_path = "/content/drive/My Drive/AIML-MRC-Capstone/models/"
project_path = "/content/drive/My Drive/AIML-MRC-Capstone/datasets/Squad2.0/TrainingDataset/"

# Create and Save Test Train Val sets - WITHOUT STOP WORDS

## Load

In [ ]:
#### NOTE THE 2 data frames's
df_nostopwords = 'squad_data_final_context_withoutstopwords.csv'
# df_withstopwords = 'squad_data_final_withstopword_withpunctuation.csv'
squad_df = pd.read_csv(project_path+'squad_data_final_context_withoutstopwords.csv')
squad_df.drop('Unnamed: 0',axis=1,inplace=True)


squad_df["answer_word_span"] = squad_df["answer_word_span"].apply(lambda x :eval(x))
print(squad_df.info())
print(squad_df['clean_context'].iloc[0])

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 130306 entries, 0 to 130305
Data columns (total 16 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   title                   130306 non-null  object 
 1   context                 130306 non-null  object 
 2   question                130306 non-null  object 
 3   id                      130306 non-null  object 
 4   answer_start            130306 non-null  int64  
 5   answer                  86807 non-null   object 
 6   plausible_answer_start  43498 non-null   float64
 7   plausible_answer        43498 non-null   object 
 8   is_impossible           130306 non-null  bool   
 9   clean_context           130306 non-null  object 
 10  clean_question          130306 non-null  object 
 11  clean_answer            130306 non-null  object 
 12  answer_len              130306 non-null  int64  
 13  answer_end              130306 non-null  int64  
 14  answer_span         

## Split 

In [ ]:
from sklearn.model_selection import train_test_split

train,test = train_test_split(squad_df,test_size = 0.2, random_state=15122011)
train,val = train_test_split(train,test_size=0.25, random_state=15122011)

print(train.shape)
print(val.shape)
print(test.shape)

(78183, 16)
(26061, 16)
(26062, 16)


## Persist

In [ ]:
train.to_csv(model_path+'train-withoutstopwords.csv')
test.to_csv(model_path+'test-withoutstopwords.csv')
val.to_csv(model_path+'val-withoutstopwords.csv')

# Build Tokenizer 

**<font color=red>BE CAREFUL WITH THIS AND DO NOT OVERWRITE</font>**

In [ ]:
from tqdm import tqdm
params['tokenizer_num_words'] = 80000
tokenizer = preprocessing.text.Tokenizer(num_words=params['tokenizer_num_words'])

# NOTE: tokenizer is been made out of original dataset
for text in tqdm([squad_df['clean_context'], squad_df['clean_question']]):  
  tokenizer.fit_on_texts(text.values)

# total tokenizer words
params['vocab_size'] = len(tokenizer.word_index)

### SAVE TOKENIZERS
with open(model_path + "tokenizer.pkl","wb") as f:
    pickle.dump(tokenizer,f)

# Create and Save Test Train Val sets - WITH STOP WORDS

## Load

In [10]:
squad_wsp_df = pd.read_csv(project_path+'squad_data_final_withstopwords_withpunctuations.csv')
squad_wsp_df.drop('Unnamed: 0',axis=1,inplace=True)


In [11]:
squad_wsp_df["answer_word_span"] = squad_wsp_df["answer_word_span"].apply(lambda x :eval(x))

In [13]:
print(squad_wsp_df.info())
print(squad_wsp_df['clean_context'].iloc[0])

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 130306 entries, 0 to 130305
Data columns (total 16 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   title                   130306 non-null  object 
 1   context                 130306 non-null  object 
 2   question                130306 non-null  object 
 3   id                      130306 non-null  object 
 4   answer_start            130306 non-null  int64  
 5   answer                  86807 non-null   object 
 6   plausible_answer_start  43498 non-null   float64
 7   plausible_answer        43498 non-null   object 
 8   is_impossible           130306 non-null  bool   
 9   clean_context           130306 non-null  object 
 10  clean_question          130306 non-null  object 
 11  clean_answer            130306 non-null  object 
 12  answer_len              130306 non-null  int64  
 13  answer_end              130306 non-null  int64  
 14  answer_span         

## Split

In [14]:
from sklearn.model_selection import train_test_split

train_wsp,test_wsp = train_test_split(squad_wsp_df,test_size = 0.2, random_state=15122011)
train_wsp,val_wsp = train_test_split(train_wsp,test_size=0.25, random_state=15122011)

print(train_wsp.shape)
print(val_wsp.shape)
print(test_wsp.shape)

(78183, 16)
(26061, 16)
(26062, 16)


## Persist

In [15]:
train_wsp.to_csv(model_path+'train-withstopwordspunct.csv')
test_wsp.to_csv(model_path+'test-withstopwordspunct.csv')
val_wsp.to_csv(model_path+'val-withstopwordspunct.csv')